<a href="https://colab.research.google.com/github/iresh-rajitha/Hello-world-To-ML-mnist/blob/main/mnist_grid_search_activation_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Data and Library

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import keras.utils
import pandas as pd
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()

# x_train = x_train.reshape((x_train.shape[0], 28, 28, 1))
# y_train = to_categorical(y_train)

x_train = tf.keras.utils.normalize(x_train, axis=1)
x_test = tf.keras.utils.normalize(x_test, axis=1)


11501568/11490434 [==============================] - 0s 0us/step


# Add one dimension to dataset

In [ ]:
image_size= 28
x_train_new = np.array(x_train).reshape(-1, image_size , image_size , 1)
x_test_new = np.array(x_test).reshape(-1, image_size , image_size , 1)
x_train_new.shape[1:]

(28, 28, 1)

# Make CNN model

In [ ]:
def create_model(neurons=16):
  model = tf.keras.models.Sequential()

  ## first cnn layer
  model.add(tf.keras.layers.Conv2D(64, (3,3), input_shape = x_train_new.shape[1:]))
  model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))
  # next cnn layers
  model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
  model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
  # flattern
  model.add(tf.keras.layers.Flatten())

  # number of dense layer 
  for x in range(2):
    model.add(tf.keras.layers.Dense(neurons, activation=tf.nn.relu))

  model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax))
  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

# model.compile(optimizer='adam',
#               loss='binary_crossentropy',
#               metrics=['accuracy'])
# model.compile(optimizer='adam',
#               loss='categorical_crossentropy',
#               metrics=['accuracy'])
# model.summary()

# Train the model

In [ ]:
# history = model.fit( x_train_new, y_train, epochs=30, validation_data=(x_test_new, y_test))
# print(history.history.keys())

# Create grid search Classifier

In [ ]:
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)

# Define Grid search parameters

In [ ]:
neurons = [512]
param_grid = dict(neurons=neurons)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, 
                    # cv=3
                    )
grid_result = grid.fit(x_train_new, y_train)

KeyboardInterrupt: ignored

# summarize results

In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))